In [1]:
from tensorflow import keras
from models.densenet import create_densenet
import tensorflow as tf
import numpy as np
from tensorflow.keras.metrics import CategoricalAccuracy
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from utils import load_config
from tensorflow.keras.preprocessing.image import load_img,img_to_array, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import pandas as pd
from sklearn.metrics import accuracy_score
from data_loader import get_loader
from tensorflow.keras.models import load_model
from models.stn import BilinearInterpolation,Localization
from models.distances import Weighted_Euclidean_Distance
from tensorflow import image

2023-02-10 17:39:30.436238: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 17:39:30.564495: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-10 17:39:31.053369: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/nariman/anaconda3/envs/tf/lib/
2023-02-10 17:39:31.053438: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugi

In [2]:
input_shape = (64,64,3)

In [3]:
encoder = create_densenet(input_shape=input_shape)

2023-02-10 17:39:32.817940: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 17:39:33.275018: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9941 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6


In [4]:
def standardize(img):
    mean = np.mean(img)
    std = np.std(img)
    return (img-mean)/std#return img /255.0

In [5]:
datapath = 'datasets/GTSRB/all'
tr_datagen = ImageDataGenerator(preprocessing_function=standardize)
train_gen = tr_datagen.flow_from_directory(datapath,batch_size=512,class_mode='categorical',target_size=(64,64))

Found 39209 images belonging to 43 classes.


In [6]:
inp = keras.layers.Input((64,64,3))
x = encoder(inp)
#x = x = keras.layers.Dense(100,activation = 'relu',kernel_initializer='he_normal')(x)
x = keras.layers.Dense(43,kernel_initializer='he_normal')(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.Activation('softmax')(x)
clf_encoder = keras.Model(inputs=inp,outputs = x)

In [7]:
clf_encoder.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 encoder (Functional)        (None, 300)               2036680   
                                                                 
 dense_1 (Dense)             (None, 43)                12943     
                                                                 
 batch_normalization_7 (Batc  (None, 43)               172       
 hNormalization)                                                 
                                                                 
 activation_7 (Activation)   (None, 43)                0         
                                                                 
Total params: 2,049,795
Trainable params: 2,042,861
Non-trainable params: 6,934
_____________________________________________

In [8]:
metric = CategoricalAccuracy()
train_loss_tracker = keras.metrics.Mean(name='loss')
#optimizer = keras.optimizers.SGD(learning_rate=0.1,momentum=0.9)
optimizer = keras.optimizers.Adam(learning_rate=0.001,epsilon=1e-8)
loss_fun = keras.losses.CategoricalCrossentropy()
clf_encoder.compile(optimizer=optimizer,loss=loss_fun,metrics=metric)

In [9]:
test_path = 'datasets/GTSRB/GTSRB_Test'
X = []
print(f'loading GTSRB Test data ...')
files = os.listdir(test_path)
files.sort()
for f in files:
    file_path = os.path.join(test_path,f)
    X += [standardize(
        img_to_array(
            load_img(
                file_path,target_size=(64,64),
                interpolation='bilinear')))]
X = tf.constant(np.array(X))
df = pd.read_csv('datasets/GTSRB/GT-final_test.csv',sep=';')
y_true = tf.constant(to_categorical(np.array(df['ClassId']),num_classes=43))

loading GTSRB Test data ...


In [11]:
best_test_acc = 0
for e in range(20):
    clf_encoder.fit(train_gen,epochs=1)
    #test_acc = test_new(clf_encoder)
    _,test_acc = clf_encoder.evaluate(X,y_true,batch_size=128,verbose=0)
    print(f'{test_acc:.4f}')
    if test_acc > best_test_acc:
        clf_encoder.save('senet_gtsrb_bench.h5')
        best_test_acc = test_acc

77/77 [==============================] - 23s 297ms/step - loss: 0.1897 - categorical_accuracy: 0.9992
0.9853
77/77 [==============================] - 23s 295ms/step - loss: 0.1341 - categorical_accuracy: 0.9999
0.9925
77/77 [==============================] - 23s 296ms/step - loss: 0.1037 - categorical_accuracy: 1.0000
0.9938
77/77 [==============================] - 23s 297ms/step - loss: 0.0838 - categorical_accuracy: 1.0000
0.9943
77/77 [==============================] - 23s 295ms/step - loss: 0.0682 - categorical_accuracy: 1.0000
0.9930
77/77 [==============================] - 23s 294ms/step - loss: 0.0583 - categorical_accuracy: 1.0000
0.9945
77/77 [==============================] - 23s 294ms/step - loss: 0.0497 - categorical_accuracy: 1.0000
0.9933
77/77 [==============================] - 23s 294ms/step - loss: 0.0423 - categorical_accuracy: 1.0000
0.9957
77/77 [==============================] - 23s 294ms/step - loss: 0.0488 - categorical_accuracy: 0.9993
0.9781
77/77 [============

In [14]:
final_h5 = 'senet_gtsrb_bench.h5'
final_model = load_model(
    final_h5,
    custom_objects={
        'BilinearInterpolation':BilinearInterpolation,
        'Localization':Localization},compile=True)

In [15]:
final_model.evaluate(X,y_true,batch_size=128,verbose=1)

99/99 [==============================] - 4s 30ms/step - loss: 0.0378 - categorical_accuracy: 0.9947


[0.037842851132154465, 0.9946951866149902]

In [18]:
from data_loader import get_loader

In [19]:
loader = get_loader('gtsrb2tt100k')

In [20]:
train_datagen, test_datagen = loader.get_generator(batch=128,dim=64)

Loading 43 all data from datasets/GTSRB
Loading 36 all data from datasets/TT100K


In [21]:
from tensorflow.keras.metrics import Mean, CategoricalAccuracy
from models.metrics import loss_mse,accuracy

In [22]:
optimizer_fn = keras.optimizers.Adam(learning_rate=0.0001)
#train_loss_tracker = keras.metrics.Mean(name='train_loss')

senet.compile(optimizer=optimizer_fn,loss=loss_mse,metrics=CategoricalAccuracy())

In [23]:
for _ in range(20):
    senet.fit(train_datagen,epochs=1)
    senet.evaluate(test_datagen,verbose=1)

  4/306 [..............................] - ETA: 33s - loss: 0.0014 - categorical_accuracy: 0.9688

KeyboardInterrupt: 